# Microsoft Machine Learning Serve

## car-service

In [ ]:
from revoscalepy import rx_lin_mod
from azureml.deploy import DeployClient
from microsoftml.datasets.datasets import get_dataset

mtcars = get_dataset('mtcars')

# -- Define the exactly same model as we did in Part 1.a --
cars_model = rx_lin_mod(formula='am ~ hp + wt', data=mtcars)

# -- Define a `code_fn` that makes a prediction using our model and test data --
def manualTransmission(hp, wt):
    import pandas as pd
    import numpy as np
    from revoscalepy import rx_predict
    from matplotlib import pyplot as plt
    
    # -- make the prediction use model `cars_model` and input data --
    new_data = pd.DataFrame({"hp":[hp], "wt":[wt]})
    answer = rx_predict(cars_model, new_data, type = 'response')
    
    # -- save arbitrary file to demonstrate the ability to return files --
    mu, sigma = 100, 15
    x = mu + sigma * np.random.randn(10000)
    hist, bins = np.histogram(x, bins=50)
    width = 0.7 * (bins[1] - bins[0])
    center = (bins[:-1] + bins[1:]) / 2
    plt.bar(center, hist, align='center', width=width)
    plt.savefig('image.png')
    
    # return prediction
    return answer

# -- authenticate against MLServer --
auth = ('{{USERNAME}}', '{{PASSWORD}}')
client = DeployClient('{{MLSERVER_HOST}}', use='MLServer', auth=auth)

# -- Publish/Deploy the `car-service`
client.service('car-service')\
   .version('v1.0.0')\
   .code_fn(manualTransmission)\
   .inputs(hp=float, wt=float)\
   .outputs(answer=pd.DataFrame)\
   .models(cars_model=cars_model)\
   .description('The car-service.')\
   .artifacts(['image.png'])\
   .deploy()

## realtime-risk-service

In [ ]:
from revoscalepy import rx_serialize_model, rx_lin_mod
from azureml.deploy import DeployClient
from microsoftml.datasets.datasets import get_dataset

attitude = get_dataset('attitude').as_df() \
   .drop('Unnamed: 0', axis = 1).astype('double')

form = "rating ~ complaints + privileges + learning + raises + critical + advance"
model = rx_lin_mod(form, attitude, method = 'regression')

s_model = rx_serialize_model(model, realtime_scoring_only=True)

# -- authenticate against MLServer --
auth = ('{{USERNAME}}', '{{PASSWORD}}')
client = DeployClient('{{MLSERVER_HOST}}', use='MLServer', auth=auth)

# -- Publish/Deploy realtime-rating-service --
client.realtime_service('realtime-rating-service') \
   .version('3.0.1') \
   .serialized_model(s_model) \
   .alias('rating') \
   .description("this is a realtime model") \
   .deploy()